In [1]:
import ScraperFC as sfc
import pandas as pd
import os

In [2]:
sofascore = sfc.Sofascore()

In [3]:
year = "2025"
liga = 'Liga 1 Peru'

In [4]:
comps = {
    # Copas internaciones de países (hombres)
    'World Cup': 16, 'Euros': 1, 'Gold Cup': 140, 'Copa America': 133,
    # Copas Continentales de clubes
    'Champions League': 7, 'Europa League': 679, 'Europa Conference League': 17015, 
    "Copa Libertadores": 384, 'Copa Sudamericana': 480,
    # Ligas europeas
    'EPL': 17, 'La Liga': 8, 'Bundesliga': 35, 'Serie A': 23, 'Ligue 1': 34, 'Turkish Super Lig': 52,
    # Ligas de AMERICA DEL SUR
    'Argentina Liga Profesional': 155, 'Argentina Copa de la Liga Profesional': 13475,
    'Liga 1 Peru': 406,  'Chile Primera Division': 11653, 'Venezuela Primera Division': 231,
    'Uruguay Primera Division': 278 , 'Ecuador LigaPro':240, 'Brasileirão Série A':325,
    # Ligas de USA
    'MLS': 242, 'USL Championship': 13363, 'USL1': 13362, 'USL2': 13546,
    # Ligas de Mexico
    'Mexico LigaMX Apertura':11621, 'LigaMX Clausura': 11620,
    # Ligas de Medio Oriente
    "Saudi Pro League": 955,
    # Copas internacionales de mujeres
    "Women's World Cup": 290
}

In [6]:
partidos = sofascore.get_match_dicts(year=year, league=liga)

Running


In [7]:
print(partidos)

[{'correctAiInsight': True, 'correctHalftimeAiInsight': False, 'tournament': {'name': 'Liga 1, Clausura', 'slug': 'liga-1-clausura', 'category': {'id': 20, 'country': {'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Peru', 'slug': 'peru'}, 'name': 'Peru', 'slug': 'peru', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'flag': 'peru', 'alpha2': 'PE', 'fieldTranslations': {'nameTranslation': {'ar': 'بيرو'}, 'shortNameTranslation': {}}}, 'uniqueTournament': {'name': 'Liga 1', 'slug': 'liga-1', 'secondaryColorHex': '#f91018', 'category': {'id': 20, 'country': {'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Peru', 'slug': 'peru'}, 'name': 'Peru', 'slug': 'peru', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'flag': 'peru', 'alpha2': 'PE', 'fieldTranslations': {'nameTranslation': {'ar': 'بيرو'}, 'shortNameTranslation': {}}}, 'userCount': 21447, 'hasPerformanceGraphFeature': False, 'id': 406, 'country': {}, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False,

In [8]:
rows = []

for partido in partidos:
    match_id = str(partido.get('id', ''))

    # URL sin request extra
    home_slug = partido.get('homeTeam', {}).get('slug', '')
    away_slug = partido.get('awayTeam', {}).get('slug', '')
    custom_id = partido.get('customId', '')
    match_url = f"https://www.sofascore.com/{home_slug}-{away_slug}/{custom_id}#id:{match_id}"

    # Campos rápidos (del listado)
    home_name   = partido.get('homeTeam', {}).get('name')
    home_id     = partido.get('homeTeam', {}).get('id')
    away_name   = partido.get('awayTeam', {}).get('name')
    away_id     = partido.get('awayTeam', {}).get('id')
    home_score  = partido.get('homeScore', {}).get('current')
    away_score  = partido.get('awayScore', {}).get('current')

    hc = partido.get('homeTeam', {}).get('teamColors', {}) or {}
    ac = partido.get('awayTeam', {}).get('teamColors', {}) or {}
    home_colors = f"Primary: {hc.get('primary','')}, Secondary: {hc.get('secondary','')}"
    away_colors = f"Primary: {ac.get('primary','')}, Secondary: {ac.get('secondary','')}"

    tournament   = partido.get('tournament', {}).get('name')
    round_number = partido.get('roundInfo', {}).get('round')
    season       = partido.get('season', {}).get('name')

    # ---- Detalle (1 request por partido) ----
    md = sofascore.get_match_dict(match_id)

    ht = md.get('homeTeam', {}) or {}
    at = md.get('awayTeam', {}) or {}
    venue_dict = (md.get('venue') or ht.get('venue') or at.get('venue') or {})  # fallback

    # Asignaciones seguras
    manager_home = (ht.get('manager') or {}).get('name')
    manager_away = (at.get('manager') or {}).get('name')

    hs = md.get('homeScore', {}) or {}
    as_ = md.get('awayScore', {}) or {}

    goles_1T_home = hs.get('period1')
    goles_2T_home = hs.get('period2')
    goles_1T_away = as_.get('period1')
    goles_2T_away = as_.get('period2')

    disp_home = hs.get('display')
    disp_away = as_.get('display')
    resultado_final = f"{disp_home} - {disp_away}" if disp_home is not None and disp_away is not None else None

    estadio = venue_dict.get('name')
    ciudad  = (venue_dict.get('city') or {}).get('name')
    fecha   = pd.to_datetime(md.get('startTimestamp'), unit='s', errors='coerce')
    arbitro = (md.get('referee') or {}).get('name')

    # Construcción de la fila (sin .update con getters anidados)
    row = {
        'match_id'          : match_id,
        'match_url'         : match_url,
        'home'              : home_name,
        'home_id'           : home_id,
        'home_score'        : home_score,
        'away'              : away_name,
        'away_id'           : away_id,
        'away_score'        : away_score,
        'home_team_colors'  : home_colors,
        'away_team_colors'  : away_colors,
        'tournament'        : tournament,
        'round_number'      : round_number,
        'season'            : season,

        # resumen detallado
        'manager_home'      : manager_home,
        'manager_away'      : manager_away,
        'goles_1T_home'     : goles_1T_home,
        'goles_2T_home'     : goles_2T_home,
        'goles_1T_away'     : goles_1T_away,
        'goles_2T_away'     : goles_2T_away,
        'resultado_final'   : resultado_final,
        'estadio'           : estadio,
        'ciudad'            : ciudad,
        'fecha'             : fecha,
        'arbitro'           : arbitro,
    }

    rows.append(row)

# Tabla final en Jupyter
df_matches = pd.DataFrame(rows).reset_index(drop=True)
df_matches

,match_id,match_url,home,home_id,home_score,away,away_id,away_score,home_team_colors,away_team_colors,...,manager_away,goles_1T_home,goles_2T_home,goles_1T_away,goles_2T_away,resultado_final,estadio,ciudad,fecha,arbitro
0,14186611,https://www.sofascore.com/club-atletico-grau-d...,Club Atlético Grau,282538,1.0,Deportivo Garcilaso,458584,1.0,"Primary: #ffffff, Secondary: #ff0000","Primary: #374df5, Secondary: #374df5",...,Carlos Bustos,0.0,1.0,0.0,1.0,1 - 1,Estadio Miguel Grau,Piura,2025-08-06 20:00:00,Víctor Cori
1,14186610,https://www.sofascore.com/deportivo-binacional...,Deportivo Binacional,275839,2.0,Alianza Atlético de Sullana,2307,0.0,"Primary: #215fba, Secondary: #feffff","Primary: #ffffff, Secondary: #6600ff",...,Gerardo Ameli,2.0,0.0,0.0,0.0,2 - 0,Estadio Guillermo Briceno Rosamedina,Juliaca,2025-08-06 22:30:00,Italo Gonzales
2,14199324,https://www.sofascore.com/comerciantes-unidos-...,Comerciantes Unidos,213609,1.0,Cienciano,2301,1.0,"Primary: #ffff00, Secondary: #ffff00","Primary: #ff0000, Secondary: #ff0000",...,Carlos Desio,0.0,1.0,0.0,1.0,1 - 1,Estadio Juan Maldonado Gamarra,Distrito de Cutervo,2025-08-08 20:00:00,Pablo Lopez Ramos
3,14199319,https://www.sofascore.com/ayacucho-alianza-lim...,Ayacucho,33894,0.0,Alianza Lima,2311,1.0,"Primary: #ffffff, Secondary: #000000","Primary: #091d47, Secondary: #ffffff",...,Nestor Gorosito,0.0,0.0,0.0,1.0,0 - 1,Estadio Ciudad de Cumaná,Ayacucho,2025-08-09 18:00:00,Micke Palomino
4,14199317,https://www.sofascore.com/asociacion-deportiva...,Asociación Deportiva Tarma,335557,1.0,Sport Huancayo,33895,0.0,"Primary: #374df5, Secondary: #374df5","Primary: #ff0000, Secondary: #ffff00",...,Richard Pellejero,0.0,1.0,0.0,0.0,1 - 0,Estadio Union Tarma,Tarma,2025-08-09 20:15:00,Augusto Menendez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,13387768,https://www.sofascore.com/club-atletico-grau-u...,Club Atlético Grau,282538,NaN,Universitario,2305,NaN,"Primary: #ffffff, Secondary: #ff0000","Primary: #ffffcc, Secondary: #ffffcc",...,Jorge Fossati,NaN,NaN,NaN,NaN,None,Estadio Miguel Grau,Piura,2025-02-23 20:30:00,None
235,13387769,https://www.sofascore.com/melgar-alianza-atlet...,Melgar,2308,2.0,Alianza Atlético de Sullana,2307,1.0,"Primary: #000000, Secondary: #ff0000","Primary: #ffffff, Secondary: #6600ff",...,Gerardo Ameli,0.0,2.0,1.0,0.0,2 - 1,Estadio de la UNSA,Arequipa,2025-02-24 00:00:00,Michael Espinoza
236,13387772,https://www.sofascore.com/deportivo-binacional...,Deportivo Binacional,275839,1.0,Cusco FC,63760,2.0,"Primary: #215fba, Secondary: #feffff","Primary: #00ffff, Secondary: #33ffff",...,Miguel Rondelli,0.0,1.0,0.0,2.0,1 - 2,Estadio Guillermo Briceno Rosamedina,Juliaca,2025-02-24 23:00:00,Julio Cesar Quiroz
237,13523645,https://www.sofascore.com/los-chankas-cyc-cien...,Los Chankas CYC,252254,NaN,Cienciano,2301,NaN,"Primary: #ffffff, Secondary: #e20e0e","Primary: #ff0000, Secondary: #ff0000",...,Carlos Desio,NaN,NaN,NaN,NaN,None,Estadio Los Chankas,Andahuaylas,2025-02-27 18:00:00,None


In [9]:
# Guardar Excel en la carpeta actual
nombre_archivo = f"Partidos_{liga}_{year}.xlsx"
df_matches.to_excel(nombre_archivo, index=False, engine='openpyxl')

# Generar enlace de descarga en Jupyter
from IPython.display import FileLink
FileLink(nombre_archivo)

c:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStatz\notebooks\Partidos_Liga 1 Peru_2025.xlsx